In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Data/Ultimate.csv')
train = pd.read_csv("Data/train.csv", parse_dates=["project_submitted_datetime"])
test = pd.read_csv("Data/test.csv", parse_dates=["project_submitted_datetime"])

#df.to_csv('Data/Ultimate.csv',index = False)

tfidf_para = {
    "sublinear_tf":True,
    "strip_accents":'unicode',
    "stop_words":"english",
    "analyzer":'word',
    "token_pattern":r'\w{1,}',
    #"ngram_range":(1,1),
    "dtype":np.float32,
    "norm":'l2',
    "min_df":5,
    "max_df":.9,
    "smooth_idf":False
}

# Thanks To
# https://www.kaggle.com/lopuhin/eli5-for-mercari
# https://www.kaggle.com/jagangupta/understanding-approval-donorschoose-eda-fe-eli5/notebook
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_col(col_name):
    return lambda x: x[col_name]

df["project_title_count"] = df["project_title"].copy()
textcols = ["text","project_resource_summary","project_title", "project_title_count","resource_description"]
vectorizer = FeatureUnion([
        ('text',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=20000,
            **tfidf_para,
            preprocessor=get_col('text'))),
        ('project_resource_summary',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=2000,
            preprocessor=get_col('project_resource_summary'))),
        ('project_title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=1500,
            preprocessor=get_col('project_title'))),
        ('project_title_count',CountVectorizer(
            ngram_range=(1, 2),
            max_features=1500,
            preprocessor=get_col('project_title_count'))),
        ('resource_description',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=2400,
            preprocessor=get_col('resource_description'))),
#         ('Non_text',DictVectorizer())
    ])

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
for c in textcols:
    df[c] = df[c].astype(str)
start_vect=time.time()
ready_df = vectorizer.fit_transform(df.to_dict('records'))
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

from sklearn.preprocessing import LabelEncoder
import gc
print('Label Encoder...')
cols = [
    'teacher_id', 
    'teacher_prefix', 
    'school_state', 
    'project_grade_category', 
    'project_subject_categories', 
    'project_subject_subcategories'
]
from tqdm import tqdm
for c in tqdm(cols):
    le = LabelEncoder()
    le.fit(df[c].astype(str))
    df[c] = le.transform(df[c].astype(str))
    print(df[c].isna().any())
del le
gc.collect()

final_test = df[df.project_is_approved.isnull()]
traindex = list(set(df.index) - set(final_test.index))
#testdex = list(set(df.index) - set(traindex))
to_drop = ['id','project_essay_1','project_essay_2','project_essay_3','project_essay_4',
           'project_resource_summary','project_submitted_datetime','project_title','resource_description',
          'text','project_is_approved',"project_title_count"]

final_test = final_test.drop(columns=to_drop)
x = df.loc[traindex,:].drop(columns=to_drop)
y = df.loc[traindex,'project_is_approved']

from scipy.sparse import hstack, csr_matrix
x = hstack([csr_matrix(x.values),ready_df[0:182080]])
final_test = hstack([csr_matrix(final_test.values),ready_df[182080:]])

x = x.tocsr()
final_test = final_test.tocsr()

#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Vectorization Runtime: 20.66 Minutes
Label Encoder...


 17%|█▋        | 1/6 [00:08<00:40,  8.19s/it]

False


 33%|███▎      | 2/6 [00:09<00:18,  4.70s/it]

...

False


 83%|████████▎ | 5/6 [00:12<00:02,  2.49s/it]

False


100%|██████████| 6/6 [00:13<00:00,  2.26s/it]

False


In [19]:
final_test = df[df.project_is_approved.isnull()]
final_test = final_test.drop(columns=to_drop)
x = df.loc[traindex,:].drop(columns=to_drop)
y = df.loc[traindex,'project_is_approved']
x = x.fillna(0)
final_test = final_test.fillna(0)
print(x.isna().any())

from scipy.sparse import hstack, csr_matrix
x = hstack([csr_matrix(x.values),ready_df[0:182080]])
final_test = hstack([csr_matrix(final_test.values),ready_df[182080:]])

x = x.tocsr()
final_test = final_test.tocsr()

project_grade_category                                                                                  False
project_subject_categories                                                                              False
project_subject_subcategories                                                                           False
school_state                                                                                            False
teacher_id                                                                                              False
teacher_number_of_previously_posted_projects                                                            False
teacher_prefix                                                                                          False
items                                                                                                   False
quantity                                                                                                False
price     

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

reg = log_reg.predict_proba(x_test)

k = pd.DataFrame(data = reg, columns=['0','1'])

fpr, tpr, _ = metrics.roc_curve(y_test, k['1'])
auc = metrics.auc(fpr, tpr)
print(auc)

0.7484147120646357


In [38]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
x_scaled = preprocessing.scale(x,with_mean = False)
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

reg = log_reg.predict_proba(x_test)

k = pd.DataFrame(data = reg, columns=['0','1'])

fpr, tpr, _ = metrics.roc_curve(y_test, k['1'])
auc = metrics.auc(fpr, tpr)
print(auc)

0.700622021384692


In [ ]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
scaler = MaxAbsScaler()
x_train, x_test, y_train, y_test = train_test_split(preprcessing.fit_transform(x), y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

reg = log_reg.predict_proba(x_test)

k = pd.DataFrame(data = reg, columns=['0','1'])

fpr, tpr, _ = metrics.roc_curve(y_test, k['1'])
auc = metrics.auc(fpr, tpr)
print(auc)